# Prompts

Prompts are essentially the instructions or questions you give to AI models to get them to generate specific responses.\
Think of them as the conversation starter that guides what the AI will say back to you.

This has even given rise to a field called "Prompt Engineering" — a discipline focused on developing and optimizing prompts for effective use of language models.

In this notebook, we'll explore prompts in Spring AI:
* Basic prompts
* Message types
* Prompting techniques

Let's add the necessary dependencies:

In [1]:
%useLatestDescriptors
%use spring-ai-anthropic

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY") ?: "YOUR_ANTHROPIC_API_KEY"

Just like in the **`Intro`** notebook, let's create `ChatOptions` and a `ChatModel`.

In [3]:
val anthropicApi = AnthropicApi.builder().apiKey(apiKey).build()
val anthropicOptions = AnthropicChatOptions.builder()
    .model(AnthropicApi.ChatModel.CLAUDE_3_5_SONNET)
    .temperature(0.7)
    .maxTokens(1024)
    .build()

val chatCompletion = AnthropicChatModel.builder()
    .anthropicApi(anthropicApi)
    .defaultOptions(anthropicOptions)
    .build()

## What makes a prompt?

A prompt is simply a text request: "tell me a joke" or "write a poem about mountains"

Let's ask our LLM to generate a haiku:

In [4]:
chatCompletion.call("Generate a hokku")

Here's a hokku (also known as haiku):

autumn leaves falling
a lone crow takes to the sky
shadows dance below

If we're using `ChatClient` and the `Prompt` class, the request would look like this:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val prompt = Prompt("Generate a hokku")
chatClient.prompt(prompt).call().content()

Here's a hokku (also known as haiku):

autumn leaves falling
a crow takes sudden flight while
shadows cross the moon

## Type of messages

In AI interactions, there are several message types (roles):
* User — message from the user
* Assistant — message from the AI
* System — instructions that guide the AI's behavior
* Tool — used for function calling

### User messages

We've been writing user messages all along.

Let's explicitly define them now:

In [6]:
val messages = Prompt(UserMessage("Generate a hokku"), UserMessage("what's name of this hokku?"))
chatClient.prompt(messages).call().content()

Here's a hokku (also known as haiku):

Morning dew sparkles
On spider's silken threads while
Autumn leaves flutter

We could call this hokku "Autumn Morning" as it captures a moment in nature during the autumn season, featuring morning dew and falling leaves.

Note: Traditional hokku follows a 5-7-5 syllable pattern and typically includes a seasonal reference (kigo) and a cutting word (kireji). This example maintains the syllable pattern and includes autumn as the seasonal reference.

### System messages

A system message tells the LLM how it should behave.
In Spring-AI, you can send a system message in several ways.

For instance, you can use a special function for `ChatClient` or create an instance of a system message and pass it to the LLM directly.

In [7]:
chatClient
    .prompt()
    .system("You are a financial expert. Answer briefly.")
    .user("If I had a time machine, should I buy Bitcoin in 2011?")
    .call()
    .content()

Yes, absolutely. Bitcoin was worth less than $1 in early 2011 and reached nearly $69,000 at its peak in 2021. Even with price fluctuations, buying and holding Bitcoin from 2011 would have yielded extraordinary returns on investment.

In [8]:
val messages = listOf(
    SystemMessage("You are a financial expert. Answer briefly."),
    UserMessage("If I had a time machine, should I buy Bitcoin in 2011?")
)

chatClient.prompt(Prompt(messages)).call().content()

Yes, absolutely. Bitcoin was worth less than $1 in early 2011 and reached nearly $69,000 at its peak in 2021. Even with price fluctuations, buying and holding Bitcoin from 2011 would have yielded astronomical returns.

### Assistant messages

As mentioned earlier, an assistant message is essentially a message from the LLM.

Let's create a simple conversation example:
we'll give the LLM a system instruction,
send a request,
and after receiving a response,
ask for clarification on a specific point.

In [9]:
val messages = mutableListOf(
    SystemMessage("You are an assistant who always answers very briefly, using no more than 10 words."),
    UserMessage("Tell me about the capital of France and its landmarks")
)
val assistantMessage = chatClient.prompt(Prompt(messages.toList())).call().chatResponse()!!.result.output
assistantMessage

AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=Paris has Eiffel Tower, Notre-Dame, and Louvre Museum., metadata={messageType=ASSISTANT}]

In [10]:
messages.add(assistantMessage)
messages.add(UserMessage("Tell me about the third landmark"))

chatClient.prompt(Prompt(messages.toList())).call().content()

Louvre Museum displays famous art, including Mona Lisa.

We've essentially created a simple dialogue between human and machine.

Notice that in the last message,
we deliberately formulated the request as:
`"Tell me about the third landmark"`.
If we hadn't sent this request along with the LLM's response about French landmarks,
we wouldn't have received a meaningful answer.

## Prompt templates

Spring AI provides PromptTemplate for working with prompts. This class uses the OSS
[String Template](https://www.stringtemplate.org/)
engine developed by Terence Parr for constructing and managing prompts.

This class allows you to use resources for prompts, making them easier to manage and localize in different languages.
It also lets you insert your data into the prompt,
which is especially useful when developing RAG applications (which we'll learn more about in future notebooks).

Let's write a simple example using PromptTemplate:

In [11]:
fun capital(): PromptTemplate {
    val message = "The two largest cities in {country}"
    return PromptTemplate(message)
}

val prompt = capital().create(mapOf("country" to "France"))
chatClient.prompt(prompt).call().content()

The two largest cities in France are:

1. Paris (population approximately 2.2 million in the city proper, over 12 million in the metropolitan area)
2. Marseille (population approximately 870,000 in the city proper, over 1.7 million in the metropolitan area)

## Prompting techniques

There are various techniques for crafting effective prompts that help us get better results from language models.
These techniques range from simple to complex and can dramatically improve the quality of AI responses.

We will consider techniques such as:
* Zero-Shot Prompting
* Few-Shot Prompting
* Chain-of-Thought (CoT) Prompting
* Meta Prompting
* Generate Knowledge Prompting
* Prompt Chaining

These are far from all the techniques. There are many more.

### Zero-Shot Prompting

Zero-shot prompting is a technique where the model performs a task based on direct instruction without being provided examples or demonstrations.
The model relies solely on its pre-training knowledge.

In [12]:
chatClient
    .prompt("""
    Classify the text as neutral, negative, or positive.
    Text: In my opinion, this restaurant is quite ordinary.
    Tonality:
    """)
    .call()
    .content()

neutral

The text "In my opinion, this restaurant is quite ordinary" expresses a neutral sentiment. The word "ordinary" suggests neither particularly good nor bad qualities, just average or standard, making this a neutral statement.

### Few-Shot Prompting

Few-shot prompting is a technique where several examples (demonstrations) of task performance are included in the prompt to help the model understand exactly how to perform a similar task.
Instead of training the model from scratch, we provide context through examples directly in the prompt.

In [13]:
chatClient
    .prompt("""
    "Zumbrik" is a small fluffy animal inhabiting the Altai Mountains. Example sentence with the word zumbrik:
    During our expedition to the Altai Mountains we met a family of cute zumbriks.

    "Fyrkotat" means to quickly rotate in one place. Example sentence with the word fyrkotat:
    """)
    .call()
    .content()

Here's an example sentence using "fyrkotat":

The excited puppy fyrkotat on the carpet when it saw its owner coming home with treats.

(Note: I understand both "zumbrik" and "fyrkotat" are made-up words being used for this exercise, and I've created a sentence that demonstrates the given meaning of "fyrkotat" - to quickly rotate in one place.)

In this example, we provided the model with one example (1-shot) of using a made-up word in a sentence.
Based on this,
the model understood the task and was able to create a similar sentence with another made-up word,
following the demonstrated pattern.

### Chain-of-Thought (CoT) Prompting

Chain-of-Thought is a prompting technique that encourages the model to show intermediate steps of reasoning before providing the final answer.
This is especially useful for complex tasks requiring mathematical calculations,
logical analysis, or multi-step reasoning.

In [14]:
chatClient
    .prompt("""
    In the group of numbers 15, 8, 3, 22, 7, 14, 26, do the odd numbers sum to an even number?
    Let's reason step by step.
    """)
    .call()
    .content()

Let me solve this step by step.

1) First, let's identify the odd numbers in the group:
   * 15 is odd
   * 3 is odd
   * 7 is odd

2) Now, let's add these odd numbers:
   * 15 + 3 + 7 = 25

3) Is 25 an even number?
   * No, 25 is odd

Therefore, the sum of the odd numbers in this group is not an even number.

The answer is no.

### Meta Prompting

Meta prompting is an advanced technique that focuses on the structural and syntactic aspects of tasks rather than specific content details.
It creates an abstract, structured way of interacting with the LLM,
where the form and pattern of information are more important than the content itself.

In [15]:
chatClient
    .prompt("""
    In problems of the format [problem P → solution S], follow this structure:
    1. Define the variables from P
    2. Construct an equation based on P
    3. Solve the equation to find S
    4. Verify the solution by substitution

    Problem: A store had x apples. After selling 15 apples and then another 1/3 of the remaining apples, the store had 20 apples left. How many apples were there initially?
    """)
    .call()
    .content()

Let me solve this step by step.

1. Define variables:
   * Let x = initial number of apples
   * 15 apples were sold first
   * 1/3 of remaining apples were sold second
   * 20 apples remained at the end

2. Construct equation:
   * After selling 15 apples: (x - 15) apples remained
   * After selling 1/3 of remaining: (x - 15) - (1/3)(x - 15) = 20
   * Simplify: (x - 15)(1 - 1/3) = 20
   * Simplify: (x - 15)(2/3) = 20

3. Solve equation:
   * (x - 15)(2/3) = 20
   * x - 15 = 20 × (3/2)
   * x - 15 = 30
   * x = 45

4. Verify solution:
   * Initial apples: 45
   * After selling 15: 45 - 15 = 30
   * After selling 1/3 of 30: 30 - (1/3 × 30) = 30 - 10 = 20
   * Final amount matches given: 20 apples

Therefore, there were 45 apples initially.

In this example, the meta prompt sets a general structure for approaching problem-solving,
not focusing on specific content but offering a universal template for analysis and solution.
The model follows this structure, applying it to the specific problem.

### Generate Knowledge Prompting

Generate Knowledge Prompting is a technique where the model first generates factual knowledge about a topic and then uses this knowledge to form a more accurate and well-founded answer.
This technique is especially useful for tasks requiring common sense or factual accuracy.

In [16]:
val knowledge = chatClient
    .prompt("""
    Request: Are lichens harmful to trees?
    Generate knowledge:
    """)
    .call()
    .content()

knowledge

Here's what you should know about lichens and their relationship with trees:

Key Facts:
1. Lichens are generally harmless to trees
- They are not parasitic
- Don't extract nutrients from trees
- Don't damage bark or wood
- Serve as indicators of good air quality

2. Benefits of lichens:
- Provide habitat for small insects
- Some birds use them for nesting material
- Can help with nitrogen fixation
- Part of healthy forest ecosystems

3. Common misconceptions:
- People often mistake lichens for tree diseases
- Presence of lichens doesn't indicate tree decline
- Lichens grow more visibly on slower-growing trees

4. When to be concerned:
- Heavy lichen growth might indicate:
  * Slow tree growth
  * Poor tree vigor
  * Need for improved growing conditions
- But lichens themselves aren't causing these issues

5. Lichen growth patterns:
- More common on older trees
- Thrive in moist environments
- Often grow on north-facing sides
- Can appear on both healthy and unhealthy trees

In [17]:
chatClient
    .prompt("""
    Answer briefly: Are lichens harmful to trees?

    knowledge: $knowledge
    """)
    .call()
    .content()

No, lichens are not harmful to trees. They are non-parasitic organisms that simply use trees as a surface to grow on, without extracting nutrients or damaging the bark or wood. While heavy lichen growth might indicate a slow-growing tree, the lichens themselves aren't causing any problems - they're actually a sign of good air quality and are part of a healthy forest ecosystem.

This technique allows the model to first gather relevant knowledge and then use it to form a more accurate and informative response.

### Prompt Chaining

Prompt Chaining is a technique where a complex task is broken down into several sequential subtasks.
The answer from one prompt becomes the input data for the next, creating a chain of operations.
This allows solving complex tasks, increases transparency, controllability, and reliability when working with LLMs.

In [18]:
val ctryCap = chatClient
    .prompt("""
    Extract all mentions of countries and their capitals from the following text.  The answer should be a list in the format "Country: Capital".

    Text:
    France is famous for the Eiffel Tower in Paris, and Germany is known for its automotive industry with headquarters in Berlin.  Meanwhile, tourists enjoy the picturesque views of Rome in Italy and the castles near Madrid in Spain.
    """)
    .call()
    .content()

In [19]:
chatClient
    .prompt("""
    Based on the following list of countries and their capitals, create a short guide to the three most interesting sights in each capital city:

    $ctryCap
    """)
    .call()
    .content()

Here's a guide to three fascinating sights in each capital city:

PARIS, FRANCE:
1. Eiffel Tower - The iconic 324-meter iron lattice tower offering spectacular city views from three observation levels
2. Louvre Museum - Home to thousands of artworks including the Mona Lisa, housed in a historic palace
3. Notre-Dame Cathedral - Magnificent Gothic cathedral (currently under restoration) known for its architecture and gargoyles

BERLIN, GERMANY:
1. Brandenburg Gate - 18th-century neoclassical monument symbolizing German unity and peace
2. East Side Gallery - The longest remaining section of the Berlin Wall, covered in historic murals
3. Museum Island - UNESCO World Heritage site hosting five world-renowned museums

ROME, ITALY:
1. Colosseum - Ancient amphitheater showcasing Roman engineering and gladiatorial history
2. Vatican Museums & Sistine Chapel - Home to masterpieces including Michelangelo's famous ceiling frescoes
3. Trevi Fountain - Baroque fountain known for its stunning sculptu

Working with prompts is fundamental for both AI application users and developers.
Spring AI and Kotlin provide a convenient and powerful API for this purpose.

Check out the next notebook to learn more about Kotlin and Spring AI!